In [1]:
import numpy as np
from numpy.random import *
from utils import mnist_reader
import random

In [2]:
dl_list = [
        'train-images-idx3-ubyte.gz',
        'train-labels-idx1-ubyte.gz',
        't10k-images-idx3-ubyte.gz',
        't10k-labels-idx1-ubyte.gz'
    ]

dataset_dir = '/Users/komei0727/workspace/robot_intelligence/data/mnist'

dataset = mnist_reader.load_mnist(dl_list, dataset_dir)
train_img,train_label,test_img,test_label = dataset

#画素値を0~1の間の値に変換
train_img = train_img / 255
test_img = test_img / 255
train_img = train_img.astype(np.float32)
#ラベルをone-hot表現で表す
train_label = [int(x) for x in train_label]
train_label_one_hot = np.identity(10)[train_label].astype(np.float32)
test_label = [int(x) for x in test_label]
test_label_one_hot = np.identity(10)[test_label]

In [3]:
noise_rate = 25

In [4]:
for i in range(len(train_img)):
    for j in range(len(train_img[0])):
        random_number = random.uniform(0,100)
        if random_number < noise_rate:
            train_img[i][j] = random.random()

In [9]:
train_img[0]

array([0.        , 0.        , 0.        , 0.        , 0.9075705 ,
       0.63868517, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.3154511 ,
       0.35866472, 0.        , 0.        , 0.        , 0.3154807 ,
       0.        , 0.        , 0.        , 0.        , 0.41156006,
       0.        , 0.        , 0.        , 0.        , 0.49635807,
       0.17632683, 0.77556837, 0.        , 0.09074488, 0.75355387,
       0.        , 0.        , 0.        , 0.        , 0.16551317,
       0.        , 0.        , 0.        , 0.41249448, 0.        ,
       0.        , 0.        , 0.9940564 , 0.        , 0.        ,
       0.        , 0.18873668, 0.        , 0.        , 0.        ,
       0.        , 0.95916545, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.59806734, 0.        ,
       0.        , 0.        , 0.        , 0.03817096, 0.48109

In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    e = np.exp(x)
    return e / np.sum(e)

def sigmoid_dash(x):
    return x * (1 - x)

def cross_entropy(y, t):
    E = 0.0
    for i in range(len(t[0])):
        E = E - t[0][i]*np.log(y[0][i])
    return E.astype(np.float32)

In [6]:
#各レイヤで共通な機能を実装
class Layer(object):
    def __init__(self, lr=0.01):
        self.params = {}
        self.grads = {}
        self.lr = lr
        
    def update(self):
        for k in self.params.keys():
            self.params[k] = self.params[k] - self.lr * self.grads[k]
            
    def zerograd(self):
        for k in self.params.keys():
            self.grads[k] = np.zeros(shape = self.params[k].shape, dtype = self.params[k].dtype)
            
#順伝播、逆伝播を行う機能を実装
class Sequential:
    def __init__(self, layers=[]):
        self.layers = layers
    #レイヤの追加
    def addlayer(self, layer):
        self.layers.append(layer)
    #順伝播の計算
    def forward(self, x):
        for l in self.layers:
            x = l.forward(x)
        return x
    #逆伝播の計算
    def backward(self, y):
        for l in reversed(self.layers):
            y = l.backward(y)
        return y
    #
    def update(self):
        for l in self.layers:
            l.update()
    #勾配をゼロに
    def zerograd(self):
        for l in self.layers:
            l.zerograd()

class LinearLayer(Layer):
    def __init__(self, input_dim, output_dim):
        super(LinearLayer, self).__init__()
        #正規分布に従った乱数による重みの初期化（Xivierの初期値）
        self.params['W'] = np.random.normal(loc = 0.0, scale = np.sqrt(1.0/input_dim), size = (input_dim, output_dim)).astype(np.float32)
        #バイアスの初期値をゼロに設定
        self.params['b'] = np.zeros(shape = (1, output_dim), dtype = np.float32)
    
    def forward(self, x):
        self.x = x
        return np.dot(x, self.params['W']) + self.params['b']
        
    def backward(self, y):
        self.grads['W'] = np.dot(self.x.T, y)
        self.grads['b'] = y
        #print(self.grads['b'])
        return y*self.params['W']
    
class SigmoidLayer(Layer):
    def __init__(self):
        super(SigmoidLayer, self).__init__()
    
    def forward(self, x):
        z = sigmoid(x)
        self.z = z
        return z
    
    def backward(self, y):
        return  np.array([np.sum(y*sigmoid_dash(self.z).T, axis=1)])     
    
class Classfier:
    def __init__(self, model):
        self.model = model
        
    def update(self, x, t):
        self.model.zerograd()
        y = self.model.forward(x)
        prob = softmax(y)
        loss = cross_entropy(prob, t)
        dout = prob - t
        dout = self.model.backward(dout)
        self.model.update()
    
    def test(self, x):
        y = softmax(self.model.forward(x))
        prob = np.array([y[0]])
        return prob

In [7]:
model = Sequential()
model.addlayer(LinearLayer(784,1000))
model.addlayer(SigmoidLayer())
model.addlayer(LinearLayer(1000,10))
classifier = Classfier(model)

In [8]:
for i in range(100):
    rand1 = np.arange(60000)
    shuffle(rand1)
    rand2 = np.arange(10000)
    shuffle(rand2)
    train_img = train_img[rand1,:]
    train_label_one_hot = train_label_one_hot[rand1,:]
    test_img = test_img[rand2]
    test_label_one_hot = np.array(test_label_one_hot[rand2,:])
    for j in range(200):
        x = np.array([train_img[j]])
        t = np.array([train_label_one_hot[j]])
        classifier.update(x, t)
    count = 0
    for j in range(100):
        x = np.array([test_img[j]])
        prob = classifier.test(x)
        if np.argmax(prob[0]) == np.argmax(test_label_one_hot[j]):
            count += 1
    print(i+1)
    print(count/100)

1
0.05
2
0.29
3
0.18
4
0.33
5
0.41
6
0.6
7
0.64
8
0.6
9
0.76
10
0.61
11
0.7
12
0.52
13
0.7
14
0.74
15
0.75
16
0.85
17
0.76
18
0.7
19
0.82
20
0.78
21
0.74
22
0.8
23
0.8
24
0.74
25
0.72
26
0.76
27
0.87
28
0.84
29
0.9
30
0.8
31
0.82
32
0.78
33
0.85
34
0.8
35
0.81
36
0.87
37
0.85
38
0.8
39
0.83
40
0.9
41
0.83
42
0.84
43
0.83
44
0.82
45
0.84
46
0.86
47
0.75
48
0.84
49
0.93
50
0.8
51
0.85
52
0.88
53
0.81
54
0.82
55
0.88
56
0.89
57
0.86
58
0.83
59
0.9
60
0.84
61
0.82
62
0.89
63
0.88
64
0.91
65
0.88
66
0.9
67
0.89
68
0.9
69
0.86
70
0.86
71
0.84
72
0.9
73
0.89
74
0.85
75
0.89
76
0.9
77
0.85
78
0.88
79
0.88
80
0.85
81
0.93
82
0.89
83
0.84
84
0.9
85
0.82
86
0.89
87
0.83
88
0.88
89
0.91
90
0.92
91
0.92
92
0.91
93
0.85
94
0.84
95
0.86
96
0.91
97
0.91
98
0.84
99
0.84
100
0.88
